### 1. Подготовка

Чтобы подготовиться к сбору данных о товарах из [онлайн-магазина Winmart](https://winmart.vn/), импортируем необходимые библиотеки и задаём ключевые параметры для работы с API: базовый URL для получения категорий, текущую дату для отслеживания и заголовки запроса.

In [ ]:
import requests
import json
import time
import random
import pandas as pd
from datetime import date

# задаём базовый URL и заголовки для запросов к сайту
CATEGORIES_URL = "https://api-crownx.winmart.vn/mt/api/web/v1/category"
CHECK_DATE = date.today() # текущая дата для маркировки файлов

HEADERS = {
    'origin': 'https://winmart.vn',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36'
}

### 2. Веб-скрейпинг

Чтобы извлечь данные о товарах с сайта Winmart, мы действуем поэтапно: загружаем список категорий, собираем данные по каждой категории, очищаем результаты и сохраняем всё в структурированном формате.

#### 2.1. Получение категорий

Начинаем с получения списка товарных категорий через API Winmart. Функция извлекает нужные поля (*code*, *name*, родительскую категорию, флаг наличия подкатегорий, *slug* — используется для построения URL, и *level*) и возвращает очищенный список.

Каждая категория определяется числовым кодом и slug-идентификатором. Числовой код не необходим для последующих запросов, а вот slug нужен для составления корректного URL запроса.

In [ ]:
def fetch_categories():
    
    response = requests.get(CATEGORIES_URL, headers=HEADERS) # получаем данные о категориях от API
    raw_categories = json.loads(response.text) # преобразуем JSON-ответ в список словарей с категориями

    cleaned_categories = []
    
    # проходим по категориям верхнего уровня и выбираем только нужные поля
    for category in raw_categories["data"]:
        cleaned_categories.append({
            "code": category["parent"]["code"],  # числовой код для последующего запроса товаров
            "name": category["parent"]["name"],  # название для справки
            "parent_id": None,   # код и имя родительской категории 
            "parent_name": None, # на верхнем уровне нет родителя, но поле добавлено для унификации
            "has_child": True if category["lstChild"] else False, # True, если у категории есть подкатегории — важно для загрузки товаров далее
            "slug": category["parent"]["seoName"], # имя категории, используемое в запросах для получения товаров
            "level": category["parent"]["level"] # уровень вложенности категории
        })
        if category["lstChild"]:    # если есть подкатегории, проходим по ним тоже
            for child in category["lstChild"]:
                cleaned_categories.append({
                    "code": child["parent"]["code"],
                    "name": child["parent"]["name"],
                    "parent_id": category["parent"]["code"],
                    "parent_name": category["parent"]["name"],
                    "has_child": False,
                    "slug": child["parent"]["seoName"],
                    "level": child["parent"]["level"]
                })

    return cleaned_categories

Мы получаем все доступные категории и сохраняем их в CSV-файл — чтобы данные было удобно просматривать и при необходимости повторно использовать. Для сбора данных о товарах используются только самые нижние категории (те, у которых нет подкатегорий) — они меньше, это снижает риск блокировки и потери данных при ошибке запроса.

In [ ]:
categories = fetch_categories()
categories_df = pd.DataFrame(categories)
categories_df.to_csv(f'categories-{CHECK_DATE}-winmart.csv', index=False)

In [ ]:
# необязательный промежуточный вывод списка категорий
# with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
#     display(categories_df)

In [ ]:
categories_end = [cat for cat in categories if not cat['has_child']] # выбираем только категории без подкатегорий
# categories_end = categories_end[:] # эту строку можно использовать, чтобы продолжить сбор данных с определённого места в случае ошибки
total_categories_count = len(categories_end)

#### 2.2. Получение исходных данных о товарах

Списки товаров получаются от API сайта, который возвращает данные порциями (страницами). Каждый запрос требует указания slug категории, номера страницы и других параметров: размера страницы (обычно 8 товаров), кода магазина и кода группы магазинов.

При заходе на страницу категории API загружает первые 8 товаров, а при прокрутке страница отправляет дополнительные запросы для получения следующей группы товаров. Первый запрос также возвращает общее количество страниц, что позволяет определить общее количество запросов для данной категории.

Мы проходим по всем страницам выбранной категории, добавляя случайную задержку между запросами, чтобы снизить риск блокировки или ограничения частоты запросов. В результате получаем «сырую» информацию о товарах в формате JSON.

In [ ]:
def fetch_products(category):
    
    products = []

    # задаём URL и параметры для запроса
    ITEMS_URL = "https://api-crownx.winmart.vn/it/api/web/v3/item/category"
    PARAMS = {
            'orderByDesc': 'true',
            'pageNumber': '1',
            'pageSize': '8', # устанавливаем равным 8 — это соответствует поведению сайта
            'slug': category["slug"],
            'storeCode': '1561',
            'storeGroupCode': '1999',
        }
    
    response = requests.get(ITEMS_URL, headers=HEADERS, params=PARAMS) # запрашиваем первую страницу товаров
    response_data = json.loads(response.text)
    total_pages = response_data["paging"]["totalPages"] # получаем общее количество страниц товаров
    
    for page in range(1, total_pages + 1):
        
        time.sleep(random.uniform(1, 5)) # случайная задержка, чтобы избежать блокировки или ограничений
        
        PARAMS = {
            'orderByDesc': 'true',
            'pageNumber': str(page),
            'pageSize': '8',
            'slug': category['slug'],
            'storeCode': '1561',
            'storeGroupCode': '1999',
        }
    
        response = requests.get(ITEMS_URL, headers=HEADERS, params=PARAMS) # получаем данные о товарах на текущей странице
        response_data = json.loads(response.text)
        products += response_data["data"]["items"] # извлекаем только товары и добавляем их в список
        
        print(f'{len(products)}..', end='') # индикатор прогресса

    return products   

#### 2.3. Очистка данных о товарах

Далее мы определяем функцию, которая извлекает из сырых данных только необходимые поля:

- Идентификатор категории (для справки)
- Название товара
- Цена
- Единица измерения
- Название супермаркета

В итоге получается очищенный список товаров, готовый к сохранению или дальнейшему анализу.

In [ ]:
def clean_product_data(category, raw_products):

    cleaned_products = []

    # проходим по сырым данным и выбираем только нужные поля
    for product in raw_products:
        cleaned_products.append({
            "category_code": category["code"],
            "name": product["name"],
            "price": product["price"],
            "uom": product["uom"],
            "supermarket": 'Winmart' # название супермаркета как источника данных
        })

    return cleaned_products

#### 2.4. Сводим все вместе

В следующей ячейке кода:

1. Проходим по выбранным категориям нижнего уровня (категориям без подкатегорий),
2. Получаем и очищаем данные по товарам каждой категории,
3. Добавляем очищенные данные в общий CSV-файл, собирая полный набор данных.

Для удобства отслеживания в процессе выводится количество загруженных товаров по каждой категории. Между итерациями добавляется небольшая случайная задержка, чтобы избежать ограничения по количеству запросов. Имя выходного файла содержит дату (`CHECK_DATE`), чтобы зафиксировать, когда данные были выгружены.

In [ ]:
fetched_categories_count = 0    # счётчик категорий для отслеживания прогресса загрузки

# создаём файл с заголовком таблицы для записи данных
products_df = pd.DataFrame(columns=["category_code", "name", "price", "uom", "supermarket"])
products_df.to_csv(f'scraped_products-{CHECK_DATE}-winmart.csv', index=False, mode='w')

for category in categories_end:
    
    raw_products = fetch_products(category) # загружаем товары
    new_products = clean_product_data(category, raw_products) # извлекаем только необходимые данные и добавляем новые товары в список

    products_df = pd.DataFrame(new_products)
    products_df.to_csv(f'scraped_products-{CHECK_DATE}-winmart.csv', index=False, mode='a', header=False)

    fetched_categories_count += 1
    print(f'Категория {category["code"]} обработана, загружено {fetched_categories_count} из {total_categories_count} категорий')
    
    time.sleep(random.uniform(1, 5))

print(f'Загрузка завершена. Результаты сохранены в файл scraped_products-{CHECK_DATE}-winmart.csv')

### 3. Фильтрация и нормализация данных о товарах

После сбора и очистки исходных данных о продуктах мы переходим к фильтрации, чтобы включить только те товары, которые мы планируем сравнивать. Этот этап состоит из нескольких шагов:

#### 3.1. Первичная обработка

Начинаем с загрузки ранее сохранённых данных с товарами и категориями.

- Удаляем поле `category_code`, так как оно не нужно для следующих шагов.
- Удаляем дублирующиеся записи товаров — они могли появиться, если один и тот же товар принадлежал к нескольким категориям.
- Также удаляем лишние пробелы в названиях товаров, чтобы привести их к единому виду.

In [ ]:
import pandas as pd
import re

categories = pd.read_csv('categories-2025-03-06-winmart.csv')
products_original = pd.read_csv('scraped_products-2025-03-06-winmart-complete.csv')

products = products_original.drop(['category_code'], axis=1)    # удаляем поле category_code
products = products.drop_duplicates() # удаляем дубли
products['name'] = products['name'].str.strip() # удаляем лишние пробелы

#### 3.2. Фильтрация товаров по типу

Чтобы выбрать из набора только релевантные для сравнения товары, мы создаём словарь с типами продуктов и соответствующими регулярными выражениями. Каждое выражение выбирает базовую форму продукта, исключая вариации, которые не входят в сферу нашего анализа.

> **Примечание:** приведенные здесь регулярные выражения не универсальны и подобраны под конкретный набор данных.

In [ ]:
product_regex_map = {
    'rice': r'^gạo(?!.*lứt)',
    'bread': r'^bánh (mì|mỳ|sandwich)(?!.*(bơ|hoa|chà|thịt))',
    'chicken_fillet': r'(fillet|phi lê|\bức)(?!.*đùi).*gà',
    'pork_leg': r'đùi.*heo',
    'egg': r'^trứng gà',
    'cucumber': r'^dưa.*(chuột|leo)(?!.*ngâm)',
    'carrot': r'^cà rốt',
    'onion': r'^hành tây',
    'tomato': r'^cà chua',
    'cabbage': r'bắp cải trắng', # выбирает "bắp cải trắng", но исключает другие разновидности, как "bắp cải tím"
    'eggplant': r'cà tím',
    'banana': r'^chuối(?!.*sấy)',
    'orange': r'^cam',
    'milk': r'^(thùng.*sữa|sữa|lốc.*sữa)(?!.*(vị|hương|có đường|ít đường|socola|sô cô la|dâu|chua|lên men|bắp|lact|yến)).*(trùng|tươi|tự nhiên)',
     # ↑ выбирает простое свежее молоко и исключает молоко с добавками или сахаром и кисломолочные продукты
    'yogurt': r'^(sữa chua|lốc.*hộp.*sữa chua)(?!.*(uống|lên men|fristi|chai))',
    'condensed_milk': r'sữa đặc',
    'black_tea': r'^trà\b(?!.*(ml|l\b|lít|sữa|gừng|nestea|atiso|ice|xanh|ô long|nhài|tôm|thái|hòa tan|zoga))',
    # ↑ выбирает чаи, но исключает все разновидности зеленого чая, а также порошковый и готовый холодный чай
    'green_tea': r'^trà\b(?!.*(ml|l\b|lít|sữa|gừng|nestea|atiso|ice|đen|hòa tan|zoga|hoa cúc|ceylon|đào))',
    'ground_coffee': r'^(cà phê|café)(?!.*hòa tan).*(bột|xay|sáng|chế phin|nâu|khát)',
    'sugar': r'^đường.*(pure|trắng|mía)',
    'salt': r'^muối.*biển', # выбирает "muối biển", но исключает соль с примесями, как, например, "muối tôm"
    'sunflower_oil': r'dầu.*hướng dương',
    'soybean_oil': r'dầu.*nành',
    'water': r'^nước (uống|khoáng|tinh)(?!.*(vị|sữa|tăng|ion))',
    'spaghetti': r'^mì(?!.*(kool|xốt)).*(ý|spag)',
    'rice_noodles': r'^bún.*safoco',
    'tofu': r'^(đậu|tàu) hũ(?!.*(chiên|trứng))',
    'water_spinach': r'^rau.*muống',
    'mango': r'^xoài(?!.*sấy)',
    'fish_sauce': r'^nước mắm(?!.*ớt)'
}
product_regex_list = '|'.join(product_regex_map.values()) # объединяем все регулярные выражения в одно, используя оператор OR (|)

# фильтруем товары, названия которых соответствуют одному из выбранных типов продуктов
filtered_products = products.loc[products.name.str.contains(product_regex_list, case=False, regex=True)]

#### 3.3. Присвоение типа продукта

Каждому отфильтрованному товару присваивается метка с типом продукта на основе совпадения по регулярному выражению.

In [ ]:
def assign_product_type(row):
    name = row['name']
    for product_type, regex in product_regex_map.items():
        match = re.search(regex, name, flags=re.IGNORECASE)
        if match:
            return product_type
    return None

filtered_products = filtered_products.copy()  # создаём копию, чтобы избежать предупреждений при добавлении новых полей через .loc
filtered_products.loc[:,'product_type'] = filtered_products.apply(assign_product_type, axis=1)

# необязательный промежуточный вывод списка отфильтрованных товаров
# with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
#     display(filtered_products)

#### 3.4. Извлечение и нормализация единиц измерения

Разные единицы товаров отличаются по количеству, весу или объёмы. Чтобы сравнение было корректным, мы извлекаем нужную информацию из названия товара или единицы измерения и рассчитываем нормализованные показатели — цену за килограмм, литр или за штуку.

- Вес в граммах
- Количество штук (в частности, яиц)
- Объём в миллилитрах

Каждое из значений извлекается по шаблону (через регулярное выражение). Не все товары содержат все эти значения, поэтому соответствующие колонки с нормализованными ценами (*price_kg*, *price_lit*, *price_unit*) могут остаться пустыми для некоторых товаров.

In [ ]:
def extract_weight(row):
    """Извлекает общий вес в граммах из названия товара или поля uom."""
    
    name, uom = row['name'], row['uom']
    
    # некоторые товары продаются комплектами, это указано в uom (например, 'G3' означает 3 упак.)
    match = re.search(r'G(\d)', uom)
    if match:
        number = int(match.group(1))
    elif uom == 'T': # если uom = 'T' (thùng — коробка), количество упаковок в коробке указывается в названии
        match = re.search(r'(\d+)\s?(gói|hộp|túi)', name, flags=re.IGNORECASE)
        if match:
            number = int(match.group(1))
    else:
        number = 1 # если информации нет — считаем, что одна штука
    # проверяем, указан ли вес в названии
    match = re.search(r'(\d+|\d+[,.]\d+)\s?(g\b|kg)', name, flags=re.IGNORECASE)
    if match:
        weight = float(match.group(1).replace(',', '.')) # извлекаем вес
        unit = match.group(2)
        return number * weight * 1000 if unit in ['kg','Kg'] else number * weight # переводим кг в граммы при необходимости
    # проверяем uom
    if uom == 'KG':
        weight = 1000
        return weight
    
    return None  # если ничего не найдено

# следующие две функции работают по той же логике, что и extract_weight, но для штук и миллилитров
def extract_number_of_units(row):
    """Извлекает количество штук из названия товара или поля uom
    (quả, trái ~ штука).
    """
    
    name, product_type, uom = row['name'], row['product_type'], row['uom']
    
    # проверяем название
    match = re.search(r'(\d+)\s?(quả|trái)', name, flags=re.IGNORECASE)
    if match:
        number_of_units = int(match.group(1))
        return number_of_units
    # для яиц берём любое число из названия (Winmart обозначает это именно так)
    match = re.search(r'(\d+)', name)
    if match and product_type == 'egg':
        number_of_units = int(match.group(1))
        return number_of_units
    
    return None  # если ничего не найдено

def extract_volume(row):
    """Извлекает объём в миллилитрах из названия товара или поля uom."""
    
    name, uom = row['name'], row['uom']
    
    # проверка на комплект упаковок
    match = re.search(r'G(\d)', uom)
    if match:
        number = int(match.group(1))
    elif uom == 'T':
        match = re.search(r'(\d+)\s?(gói|hộp|túi|chai)', name, flags=re.IGNORECASE) # (hộp - коробка, túi - пакет, chai - бутылка)
        if match:
            number = int(match.group(1))
        else:
            number = 0
    else:
        number = 1
    # ищем указание объёма в названии
    match = re.search(r'(\d+|\d+[,.]\d+)\s?(ml|l\b|lít)', name, flags=re.IGNORECASE)
    if match:
        volume = float(match.group(1).replace(',', '.'))
        unit = match.group(2)
        return number * volume * 1000 if unit in ['l', 'L', 'lít'] else number * volume

    return None  # если ничего не найдено

filtered_products = filtered_products.copy()  # пересоздаем датафрейм

# вычисляем нормализованные цены
filtered_products.loc[:,'weight'] = filtered_products.apply(extract_weight, axis=1)  # столбец с весом в граммах
filtered_products.loc[:,'price_kg'] = filtered_products.price / filtered_products.weight * 1000   # столбец с ценой за кг

filtered_products.loc[:,'number_of_units'] = filtered_products.apply(extract_number_of_units, axis=1)  # столбец с количеством штук
filtered_products.loc[:,'price_unit'] = filtered_products.price / filtered_products.number_of_units   # столбец с ценой за штуку

filtered_products.loc[:,'volume'] = filtered_products.apply(extract_volume, axis=1)  # столбец с объемом в мл
filtered_products.loc[:,'price_lit'] = filtered_products.price / filtered_products.volume * 1000   # столбец с ценой за литр

# необязательный промежуточный вывод
# with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
#     display(filtered_products)

#### 3.5. Сохранение финального отфильтрованного набора данных

Наконец, отфильтрованный и дополненный набор данных сохраняется в новый CSV-файл для дальнейшего анализа.

In [ ]:
filtered_products.to_csv(f'filtered_products-2025-03-06-winmart.csv')